In [9]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl

import datetime
today = date.today()
today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'02.20.2022'

In [4]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

shutil.move(downloads + "/"+ "DKSalaries.csv",
           contest_data_loc + "/"+ "DKSalaries.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries.csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("DKSalaries.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [5]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/DraftKings'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/vchang/Downloads/DK NBA Projections.csv'

In [10]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\DraftKings\ETR_Daily_' + str(today) + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position', 'Name + ID']], 
              left_on='Player', 
              right_on='Name')

In [11]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df = df.explode('Roster Position')

df.head()

df.to_excel("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

In [12]:
df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,Dennis Schroder,HOU,LAC,35.0,PG/SG,4400.0,32.4,8.8,41.0,67.8,MAIN,Dennis Schroder,PG,Dennis Schroder (21367802)
0,Dennis Schroder,HOU,LAC,35.0,PG/SG,4400.0,32.4,8.8,41.0,67.8,MAIN,Dennis Schroder,SG,Dennis Schroder (21367802)
0,Dennis Schroder,HOU,LAC,35.0,PG/SG,4400.0,32.4,8.8,41.0,67.8,MAIN,Dennis Schroder,G,Dennis Schroder (21367802)
0,Dennis Schroder,HOU,LAC,35.0,PG/SG,4400.0,32.4,8.8,41.0,67.8,MAIN,Dennis Schroder,UTIL,Dennis Schroder (21367802)
1,Reggie Jackson,LAC,HOU,33.5,PG,6800.0,38.8,5.6,47.2,35.2,MAIN,Reggie Jackson,PG,Reggie Jackson (21367743)


In [13]:
os.getcwd()

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\PlayerPool Analysis'

In [33]:
df = pd.read_csv("DKNG_final_PlayerPool_ETR_02.20.2022.csv", index_col=['Roster Position' + 'Name + ID'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['DK Salary'].to_dict()
values = df['DK Points'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n,p]*values[n,p] for (n,p) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n,p]*costs[n,p] for (n, p) in legal_assignments]) <= 50000

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PG']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SG']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'G']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'F']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'UTIL']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0], idx[1])
        
        result.append([idx[0], idx[1]])

ValueError: Index Roster PositionName + ID invalid

In [23]:
new_list = result
df = pd.DataFrame(new_list)
df.to_excel("a_result.xlsx")

# Create the Constraint Problem

Goal: Maximize DK Points

- Total Players = 8
- TotalSalary <= 50000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_G
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_F
- TotalPosition_C
- TotalPosition_UTIL

In [32]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [33]:
# number of players constraint
prob += pulp.lpSum(player_var for player_var in player_vars) == 8
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['Roster Position'].iloc[i]) for i in range(len(df))])
    

prob += get_position_sum(player_vars, df, 'PG') >= 1
prob += get_position_sum(player_vars, df, 'SG') >= 3
prob += get_position_sum(player_vars, df, 'G') >= 1
prob += get_position_sum(player_vars, df, 'SF') >= 1
prob += get_position_sum(player_vars, df, 'PF') >= 1
prob += get_position_sum(player_vars, df, 'F') >= 3
# prob += get_position_sum(player_vars, df, 'C') <= 2
prob += get_position_sum(player_vars, df, 'C') >= 1

# total salary constraint
prob += pulp.lpSum(df['DK Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 50000

# finally, specify the goal
prob += pulp.lpSum([df['DK Points'].iloc[i] * player_vars[i] for i in range(len(df))])

KeyError: 'Roster Position'

In [34]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

PulpError: Repeated variable names:
[('player_Alperen_Sengun', 4), ('player_Amir_Coffey', 5), ('player_Bobby_Portis', 2), ('player_Christian_Wood', 2), ('player_Danny_Green', 3), ('player_David_Nwaba', 4), ('player_Dennis_Schroder', 4), ('player_Furkan_Korkmaz', 5), ('player_Garrison_Mathews', 3), ('player_Georges_Niang', 3), ('player_Giannis_Antetokounmpo', 4), ('player_Isaiah_Hartenstein', 2), ('player_Ivica_Zubac', 2), ("player_Jae'Sean_Tate", 3), ('player_Jalen_Green', 4), ('player_Joel_Embiid', 2), ('player_Jordan_Nwora', 3), ('player_Josh_Christopher', 5), ('player_Jrue_Holiday', 3), ('player_KJ_Martin', 4), ('player_Khris_Middleton', 5), ('player_Lindell_Wigginton', 3), ('player_Luke_Kennard', 4), ('player_Marcus_Morris_Sr.', 4), ('player_Matisse_Thybulle', 4), ('player_Nicolas_Batum', 4), ('player_Paul_Millsap', 2), ('player_Paul_Reed', 4), ('player_Reggie_Jackson', 3), ('player_Robert_Covington', 4), ('player_Rodney_Hood', 5), ('player_Sandro_Mamukelashvili', 4), ('player_Semi

In [54]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
result = []
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['Roster Position'], row.Player, row.Team, row['DK Salary'], row['DK Points'], row['Name + ID'])
        total_salary_used += row['DK Salary']
        mean_AvgPointsPerGame += row['DK Points']
        result.append([row.Player, row['Roster Position'], row['Name + ID']])
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

['PG', 'SG', 'G', 'UTIL'] Dennis Schroder HOU 4400.0 32.4 Dennis Schroder (21367802)
['PG', 'G', 'UTIL'] Reggie Jackson LAC 6800.0 38.8 Reggie Jackson (21367743)
['C', 'UTIL'] Ivica Zubac LAC 4700.0 30.1 Ivica Zubac (21367874)
['SF', 'PF', 'F', 'UTIL'] Marcus Morris Sr. LAC 6200.0 34.5 Marcus Morris Sr. (21367748)
['C', 'UTIL'] Joel Embiid PHI 11800.0 56.3 Joel Embiid (21367820)
['PF', 'F', 'UTIL'] Georges Niang PHI 3200.0 20.9 Georges Niang (21367792)
['SG', 'SF', 'F', 'G', 'UTIL'] Terance Mann LAC 5100.0 28.2 Terance Mann (21367843)
['SG', 'SF', 'F', 'G', 'UTIL'] Khris Middleton MIL 7800.0 37.8 Khris Middleton (21367758)


(50000.0, 278.99999999999994)

In [109]:
print(result)

[['Lance Stephenson', ['PG', 'SF', 'F', 'G', 'UTIL'], 'Lance Stephenson (21347484)'], ['Clint Capela', ['C', 'UTIL'], 'Clint Capela (21347341)'], ['Trae Young', ['PG', 'G', 'UTIL'], 'Trae Young (21347148)'], ["D'Angelo Russell", ['PG', 'SG', 'G', 'UTIL'], "D'Angelo Russell (21347226)"], ['Jonas Valanciunas', ['C', 'UTIL'], 'Jonas Valanciunas (21347212)'], ['Devin Booker', ['SG', 'G', 'UTIL'], 'Devin Booker (21347168)'], ['Maxi Kleber', ['PF', 'C', 'F', 'UTIL'], 'Maxi Kleber (21347555)'], ['Terance Mann', ['SG', 'SF', 'F', 'G', 'UTIL'], 'Terance Mann (21347402)']]


In [98]:
new_list = result
df = pd.DataFrame(new_list)
df.to_excel("test_result.xlsx")

In [99]:
os.getcwd()

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\PlayerPool Analysis'